Crawler fro the netherlands:

In [1]:
!pip install bs4
!pip install selenium

   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/147.9 kB ? eta -:--:--
   ------------------- -------------------- 71.7/147.9 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 147.9/147.9 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.5 MB 2.2 MB/s eta 0:00:05
   - -------------------------------------- 0.4/10.5 MB 3.9 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/10.5 MB 6.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/10.5 MB 8.2 MB/s eta 0:00:02
   -------- ------------------------------- 2.3/10.5 MB 9.8 MB/s eta 0:00:01
   ----------- ---------------------------- 3.0/10.5 MB 11.2 MB/s eta 0:00:01
   ------------- -------------------------- 3.5/10.5 MB 11.3 MB/s eta 0:00:01
   --------------- ------------------------ 4.1/10.5 MB 11.4 MB/s eta 0:00:01
   ------

In [32]:
import requests
from bs4 import BeautifulSoup
import csv

# Base URL of the company list page
base_url = "https://nwba.nl"

# URL of the company list page
company_list_url = "https://nwba.nl/nl/leden"

# Send a GET request to the company list URL and get the response
response = requests.get(company_list_url)

# Parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the <a> tags inside <div class="raxo-readmore">
company_links = soup.select('div.raxo-readmore a')

# Open a CSV file for writing
csv_file = open('data/netherlands_member.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.DictWriter(csv_file, fieldnames=['Company Name', 'Description', 'Address', 'City', 'Postal Code', 'Contact Person', 'Email'])
csv_writer.writeheader()

# Extract the company information
for link in company_links:
    # Get the URL of the company page
    company_url = base_url + link['href']

    # Send a GET request to the company URL and get the response
    company_response = requests.get(company_url)

    # Parse the HTML content of the company response using BeautifulSoup
    company_soup = BeautifulSoup(company_response.content, 'html.parser')

    # Extract company description
    company_description = company_soup.find('p', class_='content-description')
    if company_description:
        company_description = company_description.text.strip()
    else:
        company_description = None

    # Extract company address details
    address_fields = company_soup.select('ul.fields-container li')
    address_info = {field.find('span', class_='field-label').text.strip(): field.find('span', class_='field-value').text.strip() for field in address_fields}

    # Extract address, city, and postal code
    company_name = address_info.get('Bedrijfsnaam:','')
    address = address_info.get('Adres:', '')
    city = address_info.get('Plaats:', '')
    postal_code = address_info.get('Postcode:', '')

    # Extract contact person and email
    contact_person = address_info.get('Contactpersoon:', '')
    email = address_info.get('E-mail:', '')

    # Write the data to the CSV file
    csv_writer.writerow({
        'Company Name': company_name,
        'Description': company_description,
        'Address': address,
        'City': city,
        'Postal Code': postal_code,
        'Contact Person': contact_person,
        'Email': email
    })

# Close the CSV file
csv_file.close()

In [35]:
import requests
from bs4 import BeautifulSoup
import csv

# Base URL of the company list page
base_url = "https://nwba.nl"

# URL of the company list page
company_list_url = "https://nwba.nl/nl/leden"

# Send a GET request to the company list URL and get the response
response = requests.get(company_list_url)

# Parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the <a> tags inside <div class="raxo-readmore">
company_links = soup.select('div.raxo-readmore a')

# Open a CSV file for writing
csv_file = open('data/netherlands_member.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.DictWriter(csv_file, fieldnames=['Company Name', 'Description', 'Address', 'City', 'Postal Code', 'Contact Person', 'Email'])
csv_writer.writeheader()

# Extract the company information
for link in company_links:
    # Get the URL of the company page
    company_url = base_url + link['href']

    # Send a GET request to the company URL and get the response
    company_response = requests.get(company_url)

    # Parse the HTML content of the company response using BeautifulSoup
    company_soup = BeautifulSoup(company_response.content, 'html.parser')

    # Extract company description
    company_description = company_soup.find('p', class_='content-description')
    if company_description:
        company_description = company_description.text.strip()
    else:
        company_description = None

    # Extract company address details
    address_fields = company_soup.select('ul.fields-container li')
    address_info = {field.find('span', class_='field-label').text.strip(): field.find('span', class_='field-value').text.strip() for field in address_fields}

    # Extract address, city, and postal code
    company_name = address_info.get('Bedrijfsnaam:','')
    address = address_info.get('Adres:', '')
    city = address_info.get('Plaats:', '')
    postal_code = address_info.get('Postcode:', '')

    # Extract contact person and email
    contact_person = address_info.get('Contactpersoon:', '')
    email = address_info.get('E-mail:', '')

    # Write the data to the CSV file
    csv_writer.writerow({
        'Company Name': company_name,
        'Description': company_description,
        'Address': address,
        'City': city,
        'Postal Code': postal_code,
        'Contact Person': contact_person,
        'Email': email
    })

# Close the CSV file
csv_file.close()

Now working on a carlwer for poland: https://h2poland.eu/en/hydrogen-investment-map/

In [9]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to crawl
url = "https://h2poland.eu/en/hydrogen-investment-map/"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all div elements with class "col-12 col-md-6 col-lg-4 my-3"
div_elements = soup.find_all("div", class_="col-12 col-md-6 col-lg-4 my-3")

# Iterate through each div element and extract the text content from child elements
for div in div_elements:
    # Extract text content from child elements within the div
    text_content = [element.get_text(strip=True) for element in div.find_all(["h3", "p", "a"])]

    # Join the text content with a newline character
    combined_text = "\n".join(text_content)

    # Print the extracted text content
    print(combined_text)

In [27]:
import pandas as pd

# Read the Excel file
df = pd.read_excel(r'C:\Users\André\Desktop\Universität\Universität\Master\Masterarbeit\webcrawler\data\poland_members.xlsx')

# Combine address information into a single column
df['Full Address'] = df[['Street', 'ZIP']].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)

# Save the updated DataFrame to the Excel file
df.to_excel(r'C:\Users\André\Desktop\Universität\Universität\Master\Masterarbeit\webcrawler\data\poland_members.xlsx')


Crawler for UK Members: https://ukhea.co.uk/industry-showcase/#

Here we only manage to get the first page, thus we need to improve it:

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# URL of the page to crawl
url = "https://ukhea.co.uk/industry-showcase/#"

# Navigate to the URL
driver.get(url)

# Wait for the page to load completely
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "elementor-column")))

# Get the HTML source of the rendered page
html_source = driver.page_source

# Save the HTML source to a file
with open("ukhea_page.html", "w", encoding="utf-8") as html_file:
    html_file.write(html_source)

In [4]:
# Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(html_source, "html.parser")

# Find all the company sections on the page
company_sections = soup.find_all("div", class_="jet-listing-grid__item")

# Loop through each company section to extract information
for company_section in company_sections:
    # Find the company name
    company_name = company_section.find("h2", class_="elementor-heading-title elementor-size-default").text.strip()

    # Find the company address
    company_address = company_section.find("span", class_="elementor-icon-list-text").text.strip()

    # Find all the services provided by the company
    services = [li.text.strip() for li in company_section.find_all("span", class_="elementor-icon-list-text")[1:]]

    # Print the extracted information
    print(f"Company Name: {company_name}")
    print(f"Address: {company_address}")
    print("Services:")
    for service in services:
        print(f"- {service}")
    print()

# Close the browser
driver.quit()

Company Name: 2G Energy
Address: Unit 1 Sycamore Court, Warrington Road, Runcorn, Cheshire, WA7 1RS
Services:

Company Name: Action Sealtite
Address: 
Services:

Company Name: Acute Civil Engineering
Address: 3 Manor Court, Salesbury Hall Road, Ribchester, PR3 3XR
Services:

Company Name: Air Products
Address: Air Products PLC, Hersham Technology Park, Molesey Road, Hersham, KT12 4RZ
Services:
- Hydrogen Production
- Fuel Handling
- Hydrogen Refuelling Infrastructure
- Stationary Applications

Company Name: Alcazar
Address: 7 Bell Yard, London, WC2A 2JR
Services:
- Sector Support Functions
- Studies

Company Name: Ames Goldsmith Ceimig
Address: Units 1-3 Smeaton Road, Wester Gourdie Industrial Estate, Dundee, Tayside, DD2 4UT
Services:

Company Name: Anglo American
Address: 17 Charterhouse St, London, EC1N 6RA
Services:
- Hydrogen Production
- Investment
- Transport Applications
- Studies
- Misc.

Company Name: Ash Group
Address: 
Services:

Company Name: ATEQ
Address: Unit 71, Washfor

Here we manage to get every company from all subpages, but it is not safed in a CSV yet:

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# URL of the page to crawl
url = "https://ukhea.co.uk/industry-showcase/#"

# Navigate to the URL
driver.get(url)

# Wait for the page to load completely
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "jet-alphabet-list")))

# Find all the letter elements
letter_elements = driver.find_elements(By.CSS_SELECTOR, "div.jet-alphabet-list__row label")

# Loop through each letter element and click it
for letter_element in letter_elements:
    # Create an ActionChains instance to perform the click action
    action = ActionChains(driver)
    action.move_to_element(letter_element).click().perform()

    # Wait for the page to update
    time.sleep(2)

    # Get the updated HTML source
    html_source = driver.page_source

    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(html_source, "html.parser")

    # Find all the company sections on the page
    company_sections = soup.find_all("div", class_="jet-listing-grid__item")

    # Loop through each company section to extract information
    for company_section in company_sections:
        # Find the company name
        company_name = company_section.find("h2", class_="elementor-heading-title elementor-size-default").text.strip()

        # Find the company address
        company_address = company_section.find("span", class_="elementor-icon-list-text").text.strip()

        # Find all the services provided by the company
        services = [li.text.strip() for li in company_section.find_all("span", class_="elementor-icon-list-text")[1:]]

        # Print the extracted information
        print(f"Company Name: {company_name}")
        print(f"Address: {company_address}")
        print("Services:")
        for service in services:
            print(f"- {service}")
        print()

# Close the browser
driver.quit()

Company Name: 2G Energy
Address: Unit 1 Sycamore Court, Warrington Road, Runcorn, Cheshire, WA7 1RS
Services:

Company Name: Action Sealtite
Address: 
Services:

Company Name: Acute Civil Engineering
Address: 3 Manor Court, Salesbury Hall Road, Ribchester, PR3 3XR
Services:

Company Name: Air Products
Address: Air Products PLC, Hersham Technology Park, Molesey Road, Hersham, KT12 4RZ
Services:
- Hydrogen Production
- Fuel Handling
- Hydrogen Refuelling Infrastructure
- Stationary Applications

Company Name: Alcazar
Address: 7 Bell Yard, London, WC2A 2JR
Services:
- Sector Support Functions
- Studies

Company Name: Ames Goldsmith Ceimig
Address: Units 1-3 Smeaton Road, Wester Gourdie Industrial Estate, Dundee, Tayside, DD2 4UT
Services:

Company Name: Anglo American
Address: 17 Charterhouse St, London, EC1N 6RA
Services:
- Hydrogen Production
- Investment
- Transport Applications
- Studies
- Misc.

Company Name: Ash Group
Address: 
Services:

Company Name: ATEQ
Address: Unit 71, Washfor

KeyboardInterrupt: 

Now we finally get to safe everything in a csv, extract all informations and format it:

In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv
from selenium.webdriver.common.action_chains import ActionChains

# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# URL of the page to crawl
url = "https://ukhea.co.uk/industry-showcase/#"

# Navigate to the URL
driver.get(url)

# Wait for the page to load completely
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "jet-alphabet-list")))

# Find all the letter elements
letter_elements = driver.find_elements(By.CSS_SELECTOR, "div.jet-alphabet-list__row label")

# Function to extract company information from the page
def extract_company_information(soup):
    company_sections = soup.find_all("div", class_="jet-listing-grid__item")
    company_data = []
    for company_section in company_sections:
        company_name = company_section.find("h2", class_="elementor-heading-title elementor-size-default").text.strip()
        company_address = company_section.find("span", class_="elementor-icon-list-text").text.strip().replace("\n", " ")
        services = [li.text.strip() for li in company_section.find_all("span", class_="elementor-icon-list-text")[1:]]
        company_data.append((company_name, company_address, services))
    return company_data

# Open a CSV file for writing
csv_file = open('data/uk_membertest.csv', mode='w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Company Name', 'Address', 'Services'])

# Loop through each letter element and click it
for letter_element in letter_elements:
    # Create an ActionChains instance to perform the click action
    action = ActionChains(driver)
    action.move_to_element(letter_element).click().perform()

    # Wait for the page to update
    time.sleep(2)

    # Get the updated HTML source
    html_source = driver.page_source

    soup = BeautifulSoup(html_source, "html.parser")
    company_data = extract_company_information(soup)

    # Handle cases where tuples have more than 3 values
    for data in company_data:
        values = list(data)
        company_name = values[0]
        company_address = values[1]
        services = ', '.join(sum([values[2], []], []))  # Flatten nested list
        csv_writer.writerow([company_name, company_address, services])

# Close the CSV file and the browser
csv_file.close()
driver.quit()

Test
Extracting information from: https://vighy.france-hydrogene.org/annuaire-des-acteurs/air-liquide-2/
Test2
<html class="no-js js" lang="fr-FR"><!--<![endif]--><head>
	<meta charset="UTF-8">
		<title>
							Page not found – France Hydrogène - France Hydrogène
					</title>
			<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
	<meta http-equiv="X-UA-Compatible" content="IE=edge">
	<meta name="viewport" content="width=device-width, initial-scale=1.0">
	<link rel="author" href="https://france-hydrogene.org/app/themes/france-hydrogene.org/humans.txt">
	<link rel="pingback" href="https://www.france-hydrogene.org/wp/xmlrpc.php">

		<link rel="icon" href="https://france-hydrogene.org/app/themes/france-hydrogene.org/static/images/favicons/favicon.ico" type="image/x-icon">
    <link rel="apple-touch-icon" sizes="180x180" href="https://france-hydrogene.org/app/themes/france-hydrogene.org/static/images/favicons/apple-touch-icon.png">
    <link rel="icon" type="image/png" siz

KeyboardInterrupt: 

Now we work on a crawler for the french company data: https://www.france-hydrogene.org/les-membres/

In [83]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv
from selenium.webdriver.common.action_chains import ActionChains

# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)
max_pages = 30
csv_filename = "france_member.csv"

def extract_company_information(member_page_soup):
    # Find relevant information on the member page
    print("Test")
    name = member_page_soup.find("h1", class_="leader-title").text.strip()
    print(name)
    # Check if representative information is available
    representative_element = member_page_soup.find("div", class_="single-leader__infos__cell").find("p")
    if representative_element:
        representative = representative_element.text.strip()
    else:
        representative = "Representative information not found"
    print(representative)

    address_h3 = member_page_soup.find("h3", string="Adresse")
    if address_h3:
        address_element = address_h3.find_next_sibling("p")
        if address_element:
            address = address_element.text.strip()
        else:
            address = "Address information not found"
    else:
        address = "Address information not found"

    print(address)


    # Return the extracted information as a dictionary
    return {
        "Name": name,
        "Representative": representative,
        "Address": address
    }
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the header row
    csv_writer.writerow(["Name", "Representative", "Address"])

    # Loop through each page
    for page_number in range(1, max_pages + 1):
        print(f"Crawling page {page_number}...")

        # Update the URL with the current page number
        if page_number == 1:
            base_url = "https://www.france-hydrogene.org/les-membres/"
            driver.get(base_url)
        else:
            current_url = f"{base_url}?currpage={page_number}"
            driver.get(current_url)

        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "card-member__link")))

        html_source = driver.page_source

        soup = BeautifulSoup(html_source, "html.parser")

        member_cards = soup.find_all("article", class_="card-member")
        # Loop through each member card and click the link
        for card in member_cards:
            print("Extracting information from member card...")
            link = card.find("a", class_="card-member__link")
            link_url = link["href"]
            print(f"Extracting information from: {link_url}")

            # Navigate to the member page
            driver.get(link_url)

            # Get the updated HTML source of the member page
            member_page_source = driver.page_source

            # Create a BeautifulSoup object for the member page
            member_page_soup = BeautifulSoup(member_page_source, "html.parser")

            # Extract company information from the member page
            company_info = extract_company_information(member_page_soup)

            # Write the extracted information to the CSV file
            csv_writer.writerow([company_info["Name"], company_info["Representative"], company_info["Address"]])


driver.quit()


Crawling page 1...
Extracting information from member card...
Extracting information from: https://vighy.france-hydrogene.org/annuaire-des-acteurs/air-liquide-2/
Test
Air Liquide
Cécilia FOUVRY-RENZI
75 quai d'Orsay
75007 PARIS
Extracting information from member card...
Extracting information from: https://vighy.france-hydrogene.org/annuaire-des-acteurs/caisse-des-depots/
Test
Caisse des Dépôts
Margot LE GUEN
72, avenue Pierre Mendès France
75914 PARIS CEDEX 13
Extracting information from member card...
Extracting information from: https://vighy.france-hydrogene.org/annuaire-des-acteurs/compagnie-nationale-du-rhone/
Test
Compagnie Nationale du Rhône
Frédéric STORCK
2 rue André Bonin
69004 LYON
Extracting information from member card...
Extracting information from: https://vighy.france-hydrogene.org/annuaire-des-acteurs/neopolia/
Test
Neopolia
Benoit LEBRET
35 avenue du Général de Gaulle 44602 Saint-Nazaire
Extracting information from member card...
Extracting information from: https://

Crawler for Spain: https://aeh2.org/members/

In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv
from selenium.webdriver.common.action_chains import ActionChains


# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# URL of the page to crawl
url = "https://aeh2.org/members/"

# Navigate to the URL
driver.get(url)

# Wait for the page to load completely
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "et_pb_portfolio_item")))

def extract_company_information(member_page_soup, url):
    # Find relevant information on the member page
    print("Test")

    # Extract other company information (address, phone, etc.)
    address_element = member_page_soup.find("div", class_="et_pb_column et_pb_column_4_4 et_pb_column_inner et_pb_column_inner_0_tb_body et-last-child")

    # Extract company name and address
    name_element = address_element.find_all("h1")
    company_name = name_element[1].get_text(strip=True)

    address_element = address_element.find_all("div", class_="et_pb_text_inner")
    address = address_element[3].get_text(strip=True)

    contact_elements = member_page_soup.find_all("div", class_="et_pb_text_inner")
    phone_number = None
    website = None
    email = None
    for contact_element in contact_elements:
        if contact_element.i and contact_element.i.get("class") == ["fas", "fa-mobile"]:
            phone_number = contact_element.i.next_sibling.strip()
        elif contact_element.i and contact_element.i.get("class") == ["fas", "fa-globe-europe"]:
            website = contact_element.i.next_sibling.strip()
        elif contact_element.i and contact_element.i.get("class") == ["fas", "fa-envelope-square"]:
            email = contact_element.i.next_sibling.strip()
    print(company_name, address, phone_number, website, email)

    return {
        "Name": company_name,
        "Address": address,
        "Phone Number": phone_number,
        "Website": website,
        "Email": email
    }

# Create a CSV file to store company information
csv_file = open("spain_member.csv", mode="w", newline="", encoding="utf-8")
csv_writer = csv.DictWriter(csv_file, fieldnames=["Name", "Address", "Phone Number", "Website", "Email"])
csv_writer.writeheader()

html_source = driver.page_source

soup = BeautifulSoup(html_source, "html.parser")

member_cards = soup.find_all("div", class_="et_pb_portfolio_item")
# Loop through each member card and click the link
for card in member_cards:
    print("Extracting information from member card...")
    link = card.find("a")
    link_url = link["href"]
    print(f"Extracting information from: {link_url}")

    # Navigate to the member page
    driver.get(link_url)
    print(link_url)
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "et_pb_row")))
    # Get the updated HTML source of the member page
    member_page_source = driver.page_source

    # Create a BeautifulSoup object for the member page
    member_page_soup = BeautifulSoup(member_page_source, "html.parser")

    # Extract company information from the member page
    company_info = extract_company_information(member_page_soup, link_url)

    # Write company information to the CSV file
    csv_writer.writerow(company_info)

driver.quit()
csv_file.close()

Extracting information from member card...
Extracting information from: https://aeh2.org/socios/abei-energy/
https://aeh2.org/socios/abei-energy/
Test
ABEI ENERGY Calle Monte Esquinza 24, 5º izquierda 655377923 https://www.abeienergy.com/es/ ivanbarahona@abeienergy.com
Extracting information from member card...
Extracting information from: https://aeh2.org/socios/abengoa-innovacion/
https://aeh2.org/socios/abengoa-innovacion/
Test
ABENGOA INNOVACIÓN C/ Energía Solar, 1 - Campus Palmas Altas, 41014 Sevilla (España) 95 493 70 00 http://www.abengoa.com/ abengoa@abengoa.com
Extracting information from member card...
Extracting information from: https://aeh2.org/socios/abo-wind-2/
https://aeh2.org/socios/abo-wind-2/
Test
ABO Wind Calle Gran Vía Marqués del Turia 25, 1º, 46005 Valencia (España) +34 963 531 180 http://www.abo-wind.com/es/ global@abo-wind.es
Extracting information from member card...
Extracting information from: https://aeh2.org/socios/mantenimiento-e-instalaciones-aclimar-sl-

Crawler for Austria: https://www.hypa.at/cluster/netzwerk

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv
from selenium.webdriver.common.action_chains import ActionChains

# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# URL of the page to crawl
url = "https://www.hypa.at/cluster/netzwerk"

# Navigate to the URL
driver.get(url)

# Wait for the page to load completely
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "col-12.mb-3.mb-md-5")))

def extract_company_information(member_page_soup):
    # Create a BeautifulSoup object for the member page
    company_name = member_page_soup.find("h1", style="margin-top: 1.5em;").get_text(strip=True)

    company_info = {}

    # Find the relevant table in the member page
    table = member_page_soup.find("table", class_="tabelle")
    if table:
        # Find all rows in the table
        table_rows = table.find_all("tr")

        # Iterate through each row
        for row in table_rows:
            # Find all columns (td tags) in the row
            cols = row.find_all("td")

            # Check if there are two columns in the row
            if len(cols) == 2:
                # Get the text from the first column (key) and second column (value)
                key = cols[0].get_text().strip()
                value = cols[1].get_text().strip()

                # Remove leading/trailing spaces and colons from the key
                key = key.replace(":", "").strip()

                # Add the key-value pair to the company_info dictionary
                company_info[key] = value
    print(company_name, company_info)
    return company_name, company_info

# Create a CSV file to store company information
csv_file = open("company_information.csv", mode="w", newline="", encoding="utf-8")
fieldnames = ["Company Name", "Additional Name", "Employees", "Legal Form", "Registration Number", "Year Established", "Member", "Street", "ZIP", "City", "Country", "Phone", "Email", "Website"]
csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
csv_writer.writeheader()

html_source = driver.page_source

soup = BeautifulSoup(html_source, "html.parser")
member_cards = soup.find_all("a", class_="btn-arrow stretched-link")

# Loop through each member card and click the link
for card in member_cards:
    print("Extracting information from member card...")
    link_url = card["href"]
    print(f"Extracting information from: {link_url}")

    # Navigate to the member page
    driver.get(link_url)

    # Get the updated HTML source of the member page
    member_page_source = driver.page_source

    # Extract company information from the member page
    company_name, company_info = extract_company_information(BeautifulSoup(member_page_source, "html.parser"))

    # Write company information to the CSV file
    csv_writer.writerow({
        "Company Name": company_name,
        "Additional Name": company_info.get("Zusatz Firmenname"),
        "Employees": company_info.get("Mitarbeiter"),
        "Legal Form": company_info.get("Rechtsform"),
        "Registration Number": company_info.get("Firmenbuchnummer"),
        "Year Established": company_info.get("Gründungsjahr"),
        "Member": company_info.get("Mitglied im Cluster"),
        "Street": company_info.get("Straße"),
        "ZIP": company_info.get("PLZ"),
        "City": company_info.get("Ort"),
        "Country": company_info.get("Land"),
        "Phone": company_info.get("Telefon"),
        "Email": company_info.get("E-mail"),
        "Website": company_info.get("Homepage")
    })

driver.quit()
csv_file.close()

Extracting information from member card...
Extracting information from: https://www.standort-tirol.at/cluster/kompetenzatlas/1765
ACONVEST GMBH {'Zusatz Firmenname': 'Styrian Hydrogen', 'Mitarbeiter': '<= 5', 'Rechtsform': 'Gesellschaft mit beschränkter Haftung', 'Firmenbuchnummer': '553127y', 'Gründungsjahr': '2021', 'Mitglied im Cluster': 'H2', 'Straße': 'Saumgasse 20', 'PLZ': '8010', 'Ort': 'Graz', 'Land': 'Österreich', 'Telefon': '+43 316 90 74 17', 'E-Mail': 'abwa@aconvest.com', 'Homepage': 'https://aconvest.com', 'Kontaktperson': 'Dr. Sari Abwa', 'Funktion': 'Geschäftsführender Gesellschafter'}
Extracting information from member card...
Extracting information from: https://www.standort-tirol.at/cluster/kompetenzatlas/1760
AA-Solutions GmbH {'Zusatz Firmenname': 'Swagelok Austria', 'Mitarbeiter': '6 - 10', 'Rechtsform': 'Gesellschaft mit beschränkter Haftung', 'Firmenbuchnummer': '457045a', 'Gründungsjahr': '2016', 'Mitglied im Cluster': 'H2', 'Straße': 'Industriezentrum NÖ-Süd St

Crawler for Italy: https://www.h2it.it/soci/

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv
from selenium.webdriver.common.action_chains import ActionChains

# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# URL of the page to crawl
url = "https://www.hypa.at/cluster/netzwerk"

# Navigate to the URL
driver.get(url)

# Wait for the page to load completely
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "jet-alphabet-list")))

# Find all the letter elements
letter_elements = driver.find_elements(By.CSS_SELECTOR, "div.jet-alphabet-list__row label")

# Function to extract company information from the page
def extract_company_information(soup):
    company_sections = soup.find_all("div", class_="jet-listing-grid__item")
    company_data = []
    for company_section in company_sections:
        company_name = company_section.find("h2", class_="elementor-heading-title elementor-size-default").text.strip()
        company_address = company_section.find("span", class_="elementor-icon-list-text").text.strip().replace("\n", " ")
        services = [li.text.strip() for li in company_section.find_all("span", class_="elementor-icon-list-text")[1:]]
        company_data.append((company_name, company_address, services))
    return company_data

# Open a CSV file for writing
csv_file = open('data/uk_membertest.csv', mode='w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Company Name', 'Address', 'Services'])

# Loop through each letter element and click it
for letter_element in letter_elements:
    # Create an ActionChains instance to perform the click action
    action = ActionChains(driver)
    action.move_to_element(letter_element).click().perform()

    # Wait for the page to update
    time.sleep(2)

    # Get the updated HTML source
    html_source = driver.page_source

    soup = BeautifulSoup(html_source, "html.parser")
    company_data = extract_company_information(soup)

    # Handle cases where tuples have more than 3 values
    for data in company_data:
        values = list(data)
        company_name = values[0]
        company_address = values[1]
        services = ', '.join(sum([values[2], []], []))  # Flatten nested list
        csv_writer.writerow([company_name, company_address, services])

# Close the CSV file and the browser
csv_file.close()
driver.quit()

Crawler for Research Institutions: https://hydrogeneuroperesearch.eu/members/?page=1

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv
from selenium.webdriver.common.action_chains import ActionChains

# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# List of page numbers to iterate over
page_numbers = range(1, 14)  # Including the last page number

member_data = []

for page_num in page_numbers:
    # URL of the page to crawl
    url = f"https://hydrogeneuroperesearch.eu/members/?page={page_num}"

    # Navigate to the URL
    driver.get(url)

    # Wait for the page to load completely
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "teaser")))

    html_source = driver.page_source

    soup = BeautifulSoup(html_source, "html.parser")

    member_cards = soup.find_all("div", class_="grid teaser")

    # Loop through each member card and navigate to the member page
    for card in member_cards:
        links = card.find_all("a", class_="teaser__block")
        for link in links:
            member_page_url = link.get("href")

            # Navigate to the member page
            driver.get(member_page_url)

            # Wait for the member page to load
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "singleMember")))

            # Get the updated HTML source of the member page
            member_page_source = driver.page_source

            # Create a BeautifulSoup object for the member page
            member_page_soup = BeautifulSoup(member_page_source, "html.parser")

            # Extract name information
            name_element = member_page_soup.find("div", class_="address").find_all("p")[0]
            name = name_element.text.strip()
            print(name)
            # Extract address information
            address_element = member_page_soup.find("div", class_="address").find_all("p")[1]
            address = address_element.text.strip()
            print(address)
            # Extract website link
            website_link = member_page_soup.find("a", class_="btn btn__tertiary").get("href")
            print(website_link)

            # Append the extracted information to the member_data list
            member_data.append({
                "name": name,
                "address": address,
                "website": website_link
            })

# Print the extracted member data
for member in member_data:
    print("Name:", member["name"])
    print("Address:", member["address"])
    print("Website:", member["website"])
    print("-" * 20)

csv_filename = "data/hydrogeneuroperesearch_member.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    fieldnames = ["name", "address", "website"]
    csv_writer = csv.DictWriter(file, fieldnames=fieldnames)
    csv_writer.writeheader()
    csv_writer.writerows(member_data)

print(f"Member data saved to {csv_filename}")

driver.quit()


Aalborg University
Fredrik Bajers Vej 7K, 9220 Aalborg, North Denmark Region Denmark
https://www.en.aau.dk/
Aalto University
Aalto University P.O. Box 11000 (Otakaari 1B) FI-00076 AALTO
https://www.aalto.fi/en
Äbo Akademi University
Tuomiokirkontori, 20500 Turku, Finland
https://www.abo.fi/en/
AGH University of Science and Technology
Aleja Adama Mickiewicza 30, 30-059 Krakow, Poland
https://www.agh.edu.pl/en/

Via Zamboni 33, 40126 Bologna, Emilia-Romagna Italy
https://www.unibo.it/it
Aragon Hydrogen Foundation (FHA)
Parque tecnológico Walqa Ctra. N-330a, 22197 Huesca, Aragon Spain
https://hidrogenoaragon.org/en/
Austrian Institute of Technology (AIT)
Giefinggasse 4, 1210, Vienna Austria
https://www.ait.ac.at/
Belgian Energy Research Alliance (BERA)
Avenue Franklin Roosevelt - Franklin Rooseveltlaan 50, Brussels, Belgium
https://energyresearchbelgium.be/en
Brandenburg University of Technology
Platz der Deutschen Einheit 1 03046 Cottbus
https://www.b-tu.de/en/
BRGM – French Geological S